# Synchronized Brainwave Dataset

## コンテキスト
EEGデバイスはより安価で目立たなくなりつつありますが、EEGデータの大きなリポジトリが少ないためなど、EEGデータを効果的に活用するアプリケーションはほとんどありません。 

UC Berkeley School of InformationのMIDSクラスは、消費者グレードの脳波検出ヘッドセットを使用して収集されたデータセットと、データの収集に使用されたソフトウェアコードおよび視覚刺激を共有しています。

データセットには、刺激提示中のすべての被験者の測定値と、刺激の開始前と終了後の測定値が含まれます。

## 内容
2つの異なるグループに2つのわずかに異なる刺激を提示しました。刺激1はここから入手でき、刺激2はこちらから入手できます。両方の刺激について、脳波データが収集されている間に、約15人のグループが刺激を同時に見ました。各人が見た刺激は、subject-metadata.csvのセッションフィールドで利用できます。 （刺激2を見た被験者は刺激1の間に部屋を去り、逆も同様です）。 stimulate-timing.csvで両方の刺激の同期時間を見つけます。各参加者について、他のメタデータも匿名で収集しました：

- （1）刺激中に表示されたビデオ（スーパーボウル広告）を以前に見たかどうか、
- （2）性別、
- （3）隠されたアイコンが表示されるかどうか
- （4）カラーカウントエクササイズ中に選択した色。

これらはすべてsubject-metadata.csvにあります。また、セッション1とセッション2の両方のすべての刺激イベントのタイミング（indra_time）を収集しました。これらの時間はstimulation-times.csvに含まれています。サーバーは、各Mindwave Mobileデバイスから毎秒1つのデータパケットを受信し、1つの行エントリにデータを保存します。

## 謝辞
このデータセットまたはソフトウェアコードまたは刺激ファイルを使用して研究結果を公開する場合は、John Chuang、Nick Merrill、Thomas Maillart、およびUC Berkeley Spring 2015 MIDS Immersion Classの学生を引用してください。 「一般的な視聴覚刺激で提示されたグループからの同期脳波記録（2015年5月9日）。」 2015年5月。

## データ分析
必要になるライブラリ呼ぶ

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

from collections import Counter

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# データ読み込み

In [2]:
train_df = pd.read_csv('~/Dropbox/@00_input/Synchronized_Brainwave/eeg-data.csv')

データフレームのカラムを取得

In [3]:
print(train_df.columns.values)

['Unnamed: 0' 'id' 'indra_time' 'browser_latency' 'reading_time'
 'attention_esense' 'meditation_esense' 'eeg_power' 'raw_values'
 'signal_quality' 'createdAt' 'updatedAt' 'label']


In [5]:
print("train data size", train_df.shape)

train data size (4812, 29)


In [4]:
train_df.head()

,Unnamed: 0,id,indra_time,browser_latency,reading_time,attention_esense,meditation_esense,eeg_power,raw_values,signal_quality,createdAt,updatedAt,label
0,3730,12,2015-05-09 23:13:42.281,1461,2015-05-09 16:13:40.954,0,0,"[944412.0, 111373.0, 52404.0, 28390.0, 3237.0,...","[-203.0, -202.0, -196.0, -185.0, -163.0, -137....",200,2015-05-09 23:13:39.550,2015-05-09 23:13:39.549+00,unlabeled
1,3732,12,2015-05-09 23:13:43.186,1461,2015-05-09 16:13:41.964,0,0,"[1793049.0, 89551.0, 3896.0, 21727.0, 9301.0, ...","[104.0, 134.0, 128.0, 121.0, 145.0, 151.0, 123...",200,2015-05-09 23:13:40.559,2015-05-09 23:13:40.559+00,unlabeled
2,3734,12,2015-05-09 23:13:44.392,1461,2015-05-09 16:13:42.950,0,0,"[400192.0, 640624.0, 153087.0, 69733.0, 98854....","[2002.0, 2047.0, 2047.0, 2047.0, 2047.0, 2047....",200,2015-05-09 23:13:41.549,2015-05-09 23:13:41.549+00,unlabeled
3,3735,12,2015-05-09 23:13:45.297,1461,2015-05-09 16:13:43.935,0,0,"[681192.0, 138630.0, 67891.0, 26459.0, 592240....","[1287.0, 1241.0, 1196.0, 1155.0, 1113.0, 1072....",200,2015-05-09 23:13:42.532,2015-05-09 23:13:42.532+00,unlabeled
4,3737,12,2015-05-09 23:13:46.201,1460,2015-05-09 16:13:44.936,0,0,"[268406.0, 197772.0, 190654.0, 266433.0, 91683...","[1905.0, 1836.0, 1770.0, 1707.0, 1645.0, 1587....",200,2015-05-09 23:13:43.532,2015-05-09 23:13:43.532+00,unlabeled


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30013 entries, 0 to 30012
Data columns (total 13 columns):
Unnamed: 0           30013 non-null int64
id                   30013 non-null int64
indra_time           30013 non-null object
browser_latency      30013 non-null object
reading_time         30013 non-null object
attention_esense     30013 non-null int64
meditation_esense    30013 non-null int64
eeg_power            30013 non-null object
raw_values           30013 non-null object
signal_quality       30013 non-null int64
createdAt            30013 non-null object
updatedAt            30013 non-null object
label                30013 non-null object
dtypes: int64(5), object(8)
memory usage: 3.0+ MB


In [6]:
def num_missing(x):
  return sum(x.isnull())

#Applying per column:
print ("Missing values per column:")
print (train_df.apply(num_missing, axis=0) )#axis=0 defines that function is to be applied on each column

Missing values per column:
Unnamed: 0           0
id                   0
indra_time           0
browser_latency      0
reading_time         0
attention_esense     0
meditation_esense    0
eeg_power            0
raw_values           0
signal_quality       0
createdAt            0
updatedAt            0
label                0
dtype: int64
